## Install a Drive FUSE wrapper, authenticate, and mount a Drive filesystem

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [11]:
cd drive


/content/drive/Toxic Analysis/drive


In [14]:
cd ..

/content/drive/Toxic Analysis


In [0]:
import pandas as pd
train=pd.read_csv("train.csv")

In [0]:
test=pd.read_csv("test.csv")

In [17]:
!pip install keras

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [0]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [0]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier

In [0]:
embed_size = 128
x = Embedding(max_features, embed_size)(inp)

In [0]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)

In [0]:
x = GlobalMaxPool1D()(x)

In [0]:
x = Dropout(0.1)(x)

In [0]:
x = Dense(50, activation="relu")(x)

In [0]:
x = Dropout(0.1)(x)

In [0]:
x = Dense(6, activation="sigmoid")(x)

In [0]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [32]:
batch_size = 32
epochs = 2
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1513s 11ms/step - loss: 0.0633 - acc: 0.9787 - val_loss: 0.0492 - val_acc: 0.9820=>.] - ETA: 0s - loss: 0.0633 - acc: 0.978
Epoch 2/2
143613/143613 [==============================] - 1524s 11ms/step - loss: 0.0448 - acc: 0.9833 - val_loss: 0.0466 - val_acc: 0.9824=>.] - ETA: 0s - loss: 0.0449 - acc: 0.983


In [0]:
pred=model.predict(X_te)

In [34]:
pred.shape

(153164, 6)

In [35]:
pred[0]

array([0.99729544, 0.2548851 , 0.96128297, 0.03863807, 0.88447636,
       0.1455345 ], dtype=float32)

In [0]:
from keras.backend import argmax
pred1=argmax(pred,axis=1)

In [37]:
pred1[0]

<tf.Tensor 'strided_slice:0' shape=() dtype=int64>

In [0]:
from keras.backend import eval
pred1=eval(pred1)

In [42]:
pred1[0]

0

In [43]:
pred1.shape

(153164,)

In [0]:
from keras.utils import np_utils

In [0]:
pred2=np_utils.to_categorical(pred1)

In [52]:
pred2.shape

(153164, 3)

In [53]:
pred.shape

(153164, 6)

In [56]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 2.3+ MB


In [57]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [63]:
pred[:][0]

array([0.99729544, 0.2548851 , 0.96128297, 0.03863807, 0.88447636,
       0.1455345 ], dtype=float32)

In [0]:
my_submission = pd.DataFrame({'id':test["id"] , 'toxic':pred[:,0],'severe_toxic':pred[:,1],'obscene':pred[:,2],'threat':pred[:,3],'insult':pred[:,4],'identity_hate':pred[:,5]})

In [0]:
my_submission.to_csv('submission.csv', index=False)